# LOGIN IN USSING HUGGINGFACE

In [1]:
from huggingface_hub import login

write_key=''
login(write_key)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

# **LOAD model and do some simple inference through pipeline**

In [3]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    # The quantization line
    model_kwargs={"torch_dtype": torch.bfloat16, "load_in_4bit": True}
)
messages = [
    {"role": "system", "content": "Answer questions"},
    {"role": "user", "content": "tell me about hercules"},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):],end='.')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Hercules, also known as Heracles in Greek mythology, is a legendary hero known for his incredible strength, courage, and accomplishments. Here are some key facts about Hercules:

**Origin:** Hercules was the son of Zeus, the king of the gods, and the mortal woman Alcmene. His birth was a result of Zeus' love for Alcmene, who was already pregnant with her husband's child. This led to the goddess Hera, Zeus' wife, to become jealous and bitter towards Hercules.

**Twelve Labors:** Hercules was tasked with completing twelve impossible labors set by King Eurystheus, who had been given the power to dictate the hero's fate. These labors included:

1. Slaying the Nemean Lion
2. Slaying the Lernaean Hydra
3. Slaying the Ceryneian Stag
4. Capturing the Erymanthian Boar
5. Cleaning the Augean Stables
6. Slaying the Mares of Diomedes
7. Capturing the Cretan Bull
8. Stealing the Mares of Diomedes
9. Obtaining the girdle of Hippolyta
10. Capturing Cerberus.

# **THE IDEA IS TO START WITH A USER PROMPT -> GENERATE OUTPUT (25 TOKENS) -> Pass this generated output as new prompt and Repeat (llama 3 talks to itself/ can easily be adapted to user talks to llama 3) . Please feel free to TRY Various Temperatures for some fun**

I have interrupted the output as free tier colab has ram limitaations

In [4]:
prompt = "Let us write an essay"  # Starting prompt
terminators = [50256]  # end-of-sequence token ID
temp=0
while True:
    # Generate outputs using the pipeline
    temp=temp+0.1
    outputs = pipeline(
        prompt,                     # The initial prompt to start text generation
        max_new_tokens=25,          # Maximum number of tokens to generate
        eos_token_id=terminators,   # End-of-sequence token IDs
        do_sample=True,             # Enable sampling to generate diverse outputs
        temperature=temp,
        top_p=0.9,
    )

    # Extract the newly generated text
    generated_text = outputs[0]["generated_text"][len(prompt):]

    # Print the generated text and append a period at the end
    print(generated_text, end='.')

    # Prepare the generated text as a user message for further processing
    systeminput = generated_text
    messages = [
        # The system message can be added here if needed
        # {"role": "system", "content": "Answer questions"},
        {"role": "user", "content": systeminput},
    ]

    # Update the prompt with the latest generated text
    prompt = systeminput



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 on the importance of education in 500 words.
Education is the backbone of any society. It is the key to unlocking the.

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 potential of individuals, communities, and nations. In today's fast-paced world, education plays a crucial role in shaping the minds.

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...
It 'labor in a foreign environment for " he. It be not your...
Then also in time "he that.

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 seeks salvation, I. This has the beginning with salvation: to ask not whether is...
(Continutione)... of ".

KeyboardInterrupt: 